# Get CDS seqs and prepare for CpG analysis



It uses the longest mRNA per gene. Different ways to obtain it. Given I had the seqeuences in a SQL databse I did it using SQL query shown in this script: https://github.com/guillemylla/Gryllus_genome_annotation/blob/master/GBI_Genome_v3/Protein_coding_genes/2-GbiV3_Functional_Annotation.md:




## Copy longest isoform

In [ ]:
# !mkdir mRNA_Seqs
# !find /NCBI_Genomes -type f -name "*longest_isoform_mrna.fa"  -exec cp '{}' mRNA\_Seqs  \;
# !cp Cricket_genome_annotation/GBI_Genome_v3/Protein_coding_genes/Annotation_Files/Longest_Prot_per_gene_mrna_V2.fa  mRNA_Seqs/Gryllus_bimaculatus_longest_isoform_mrna.fa
# !cp ~/Cricket_genome_annotation/Laupala_kohalensis_annotation/Protein_coding_genes/Annotation_Files/Lko_Longest_CDS_per_gene.fa  mRNA_Seqs/Laupala_kohalensis_longest_isoform_mrna.fa
# !cp ~/Genomes/Locusta_genome_V2.4.1/Lmi_mRNA.fa  mRNA_Seqs/Locusta_migratoria_longest_isoform_mrna.fa
# !cp ~/Genomes/Bger/i5k_ogs_v1.2.1/Bger_longest_isoform_mRNA.fa  mRNA_Seqs/Blattella_germanica_longest_isoform_mrna.fa



In [ ]:
#!cp ~/Genomes/Dmel_ncbi/Drosophila_melanogaster_longest_CDS.fa  mRNA_Seqs/

#!cp ~/Genomes/Apis_mellifera/Apis_mellifera_longest_CDS.fa  mRNA_Seqs/

#!cp ~/Genomes/Tribolium_castaneum_ncbi/Tribolium_castaneum_longest_CDS.fa mRNA_Seqs/

#!cp ~/Genomes/Bombyx_mori/Bombyx_mori_longest_CDS.fa  mRNA_Seqs/



# Count CpGoe

First count freq of C, G and CpG for each gene.

For each Species, will create 1 specific file, plus will join it to a big files woth all species.


In [ ]:
#16 insects in paper
dictionary_spp= {
         "Drosophila_melanogaster" : "Dme" ,
         "Bombyx_mori" : "Bmo" ,
         "Tribolium_castaneum" :"Tca" ,
         "Apis_mellifera" : "Ame" ,
         "Pediculus_humanus_corporis" : "Phu" ,
         "Frankliniella_occidentalis" : "Foc",
         "Laodelphax_striatella" : "Lst",
         "Nilaparvata_lugens" : "Nlu",
         "Acyrthosiphon_pisum" : "Api",
         "Sipha_flava": "Sfl",
         "Blattella_germanica" : "Bge",
         "Cryptotermes_secundus" : "Cse",
         "Zootermopsis_nevadensis" : "Zna",
         "Gryllus_bimaculatus" : "Gbi",
         "Laupala_kohalensis" : "Lko",
         "Locusta_migratoria" : "Lmi" 
}

len(dictionary_spp)

16

Function that counts CpG of all seqs form fasta file

In [ ]:
def getCpGoe(seqsfile, spp, sppAbreb, outfile1, outfile2):
    print(seqsfile)
    for record in SeqIO.parse(seqsfile, "fasta"):
        Len=len(record.seq)
        Ccount=record.seq.upper().count("C")
        Gcount=record.seq.upper().count("G")
        CGcount=record.seq.upper().count("CG")

        Cfreq=Ccount/Len
        Gfreq=Gcount/Len
        CGfreq=CGcount/(Len-1) #-1  becuase number of pairs=len-1

        if ((Cfreq*Gfreq) == 0):
            print(record.id, CGcount, record.seq, "ZEROOO")# can't devide by zero
            CGoe="NA"
        else:
            CGoe=round(CGfreq/(Cfreq*Gfreq),  7)

        print(sppAbreb, record.id,Len,round(Gfreq, 7),round(Cfreq, 7),round(CGfreq, 7), CGoe )
        
        outfile1.write(spp+"\t"+sppAbreb+"\t"+record.id+"\t"+str(Len)+"\t"+str(round(Gfreq, 7))+"\t"+str(round(Cfreq, 7))+"\t"+str(round(CGfreq, 7))+"\t"+str(CGoe)+"\n")
        outfile2.write(spp+"\t"+sppAbreb+"\t"+record.id+"\t"+str(Len)+"\t"+str(round(Gfreq, 7))+"\t"+str(round(Cfreq, 7))+"\t"+str(round(CGfreq, 7))+"\t"+str(CGoe)+"\n")


In [ ]:
from Bio import SeqIO
import os

seqs_dir=os.path.join(os.getcwd(),"mRNA_Seqs")


acomulative_outfile= open("CpGoe_outputs/All_insects_CpGpe", "w+")
acomulative_outfile.write("Spp"+"\t"+"Abb"+"\t"+"ID"+"\t"+"Len"+"\t"+"Gfreq"+"\t"+"Cfreq"+"\t"+"CGfreq"+"\t"+"CpGoe"+"\n")

for file in os.listdir(seqs_dir):
    spp=file.split("_longest")[0]
    
    ## I only want to ru CpG for species that are in dic. Not all...
    if spp in dictionary_spp: 
        sppAbreb=dictionary_spp[spp]  
        spp_outfile= open("CpGoe_outputs/"+spp+"_longest_CDS_CpGoe.csv", "w+")
        spp_outfile.write("Spp"+"\t"+"Abb"+"\t"+"ID"+"\t"+"Len"+"\t"+"Gfreq"+"\t"+"Cfreq"+"\t"+"CGfreq"+"\t"+"CpGoe"+"\n")
        # call fucntion to count CpGoe
        print(file)
        getCpGoe(os.path.join(seqs_dir, file), spp, sppAbreb, spp_outfile, acomulative_outfile)

        spp_outfile.close()
        
acomulative_outfile.close()